In [1]:
# estructured print the folder structure of the document directory. its own folder, the pared folder and al subfolders and documents

import os

def print_filtered_folder_structure(root_path="."):
    """
    Prints a readable folder structure for the given root path, including only relevant directories and files.

    Args:
        root_path (str): The path to the folder whose structure needs to be printed. Defaults to the current directory.
    """
    print(f"\n--- Folder Structure for: {os.path.abspath(root_path)} ---\n")
    for dirpath, dirnames, filenames in os.walk(root_path):
        # Skip hidden directories and files
        dirnames[:] = [d for d in dirnames if not d.startswith(".")]
        filenames = [f for f in filenames if not f.startswith(".")]

        # Print current directory
        level = dirpath.replace(root_path, "").count(os.sep)
        indent = " " * 2 * level
        print(f"{indent}[{os.path.basename(dirpath) or root_path}]")

        # Print files
        subindent = " " * 2 * (level + 1)
        for fname in filenames:
            print(f"{subindent}{fname}")

# Usage example
print_filtered_folder_structure(".")

# now with the parect folder
print_filtered_folder_structure("..")



--- Folder Structure for: c:\Users\User\Documents\Projects\Side_quest_gameoflife\src ---

[.]
  dithering.py
  game_of_life.py
  image_processing.py
  TESTING_NOTEBOOK.ipynb
  test_game_of_life.py
  utils.py
  visualization.py
  __init__.py
  [__pycache__]
    dithering.cpython-312.pyc
    game_of_life.cpython-312.pyc
    test_game_of_life.cpython-312.pyc
    visualization.cpython-312.pyc
    __init__.cpython-312.pyc

--- Folder Structure for: c:\Users\User\Documents\Projects\Side_quest_gameoflife ---

[..]
  environment.yml
  game_of_life.gif
  requirements.txt
  setup.py
  [data]
  [docs]
    algorithms.md
    README.md
  [notebooks]
    analysis.ipynb
    development.ipynb
    game_of_life.gif
    game_of_life_output.gif
    game_of_life_stats.json
    SideQuest_GameOfLife_FirstFileOfTheProject.ipynb
    VertebralNotebook.ipynb
    [results]
      game_of_life_stats.json
  [project_tests]
    test_dithering.py
    test_game_of_life.py
    __init__.py
    [__pycache__]
      test_ga

In [2]:
import os
import sys
import numpy as np
import game_of_life
# from src.visualization import visualize_game
import random

# # Asegurar que /src esté en el sys.path para los imports
# src_path = os.path.abspath(os.path.join(os.path.dirname(__file__), "src"))
# if src_path not in sys.path:
#     sys.path.append(src_path)

def generate_random_board(rows, cols, seed=None):
    """
    Genera un tablero aleatorio de dimensiones especificadas.

    Args:
        rows (int): Número de filas del tablero.
        cols (int): Número de columnas del tablero.
        seed (int, optional): Semilla para reproducibilidad.

    Returns:
        numpy.ndarray: Tablero inicial aleatorio.
    """
    np.random.seed(seed)
    print(f"Generando un tablero aleatorio de {rows}x{cols} con semilla {seed}.")
    return np.random.randint(2, size=(rows, cols))

def generate_preset_board(preset=None, rows=5, cols=5):
    """
    Genera tableros predefinidos basados en patrones clásicos o selecciona uno aleatoriamente.

    Args:
        preset (str, optional): Nombre del patrón ("block", "blinker", "glider", "random"), o None para seleccionar "random".
        rows (int, optional): Número de filas para el tablero aleatorio.
        cols (int, optional): Número de columnas para el tablero aleatorio.

    Returns:
        numpy.ndarray: Tablero inicial con el patrón seleccionado.
    """
    patterns = {
        "block": np.array([
            [1, 1],
            [1, 1]
        ]),
        "blinker": np.array([
            [0, 1, 0],
            [0, 1, 0],
            [0, 1, 0]
        ]),
        "glider": np.array([
            [0, 1, 0],
            [0, 0, 1],
            [1, 1, 1]
        ]),
        "random": lambda: generate_random_board(rows, cols)  # Generar tablero aleatorio con dimensiones especificadas
    }

    if preset is None:
        preset = "random"
        print(f"Preset no especificado. Seleccionando: {preset}.")

    print(f"Seleccionando un patrón predefinido: {preset}.")

    if preset not in patterns:
        raise ValueError(f"Preset '{preset}' no está definido.")

    if preset == "random":
        return patterns[preset]()  # Genera dinámicamente un tablero aleatorio con las dimensiones dadas

    # Ajustar el patrón predefinido al tamaño especificado (centrado en el tablero)
    pattern = patterns[preset]
    board = np.zeros((rows, cols), dtype=int)
    start_row = (rows - pattern.shape[0]) // 2
    start_col = (cols - pattern.shape[1]) // 2
    board[start_row:start_row + pattern.shape[0], start_col:start_col + pattern.shape[1]] = pattern
    return board

def run_simulation(initial_state, steps=10):
    """
    Ejecuta una simulación del Game of Life.

    Args:
        initial_state (numpy.ndarray): Tablero inicial.
        steps (int): Número de pasos a ejecutar.

    Returns:
        GameOfLife: Objeto del juego tras la simulación.
    """
    print(f"Ejecutando simulación con {steps} pasos.")
    game = GameOfLife(initial_state=initial_state, steps=steps)
    game.run()
    return game

def test_simulation(rows=None, cols=None, steps=None, preset=None, seed=None, visualize=False):
    """
    Ejecuta una simulación unitaria con parámetros ajustables o aleatorizados y reporta resultados.

    Args:
        rows (int, optional): Número de filas del tablero.
        cols (int, optional): Número de columnas del tablero.
        steps (int, optional): Número de pasos a ejecutar.
        preset (str, optional): Patrón predefinido ("block", "blinker", "glider", "random") o None para seleccionar "random".
        seed (int, optional): Semilla para reproducibilidad.
        visualize (bool, optional): Si True, genera una visualización.

    Returns:
        dict: Resumen de la simulación.
    """
    # Aleatorizar parámetros si no se proporcionan
    rows = rows or random.choice(range(10, 101))
    cols = cols or random.choice(range(10, 101))
    steps = steps or random.choice(range(1, 501))

    if preset is None:
        preset = "random"
        print(f"Preset no especificado. Seleccionando: {preset}")
        initial_state = generate_preset_board(preset, rows, cols)
    elif preset == "random":
        seed = seed or random.randint(0, 1000000)  # Generar semilla aleatoria si no se proporciona
        initial_state = generate_random_board(rows, cols, seed)
    else:
        initial_state = generate_preset_board(preset, rows, cols)

    game = run_simulation(initial_state, steps)

    if False: #visualize:
        print("Iniciando visualización...")
        visualize_game(game_of_life=game)

    summary = {
        "Dimensions": f"{rows} x {cols}",
        "Steps Executed": f"{game.execution.step_count}/{steps}",
        "Execution Time": f"{game.execution.execution_time:.2f} seconds",
        "Max Alive Cells": game.execution.max_alive_cells,
        "Min Alive Cells": game.execution.min_alive_cells,
        "Loop Detected": "Yes" if game.execution.loop_detected else "No",
        "Loop Length": game.execution.loop_length if game.execution.loop_detected else "N/A",
        "Seed": seed,
    }

    return summary


In [3]:
import pygame
import numpy as np
import random

def generate_color_pair():
    """
    Genera un par de colores predefinidos de una biblioteca estilística.

    Returns:
        tuple: (alive_color, dead_color) en formato RGB.
    """
    color_palettes = [
        ((0, 255, 0), (0, 0, 0)),  # Verde vivo sobre negro
        ((0, 0, 255), (10, 10, 30)),  # Azul vivo sobre azul oscuro
        ((255, 165, 0), (30, 15, 5)),  # Naranja sobre marrón oscuro
        ((255, 0, 0), (50, 0, 0)),  # Rojo sobre rojo oscuro
        ((255, 255, 255), (0, 0, 0)),  # Blanco sobre negro
        ((255, 223, 186), (34, 34, 59))  # Tono pastel sobre azul profundo
    ]
    return random.choice(color_palettes)

def invert_colors(alive_color, dead_color):
    """
    Invierte los colores vivos y muertos.

    Args:
        alive_color (tuple): Color actual de las celdas vivas.
        dead_color (tuple): Color actual de las celdas muertas.

    Returns:
        tuple: Nuevos colores invertidos (alive_color, dead_color).
    """
    return dead_color, alive_color

def visualize_game(
    game_of_life,
    cell_size=None,
    alive_color=None,
    dead_color=None,
    grid_color=(50, 50, 50),
    fps=None,
    show_stats=False
):
    """
    Visualiza la simulación del Game of Life usando pygame, con detección de bucles largos.

    Args:
        game_of_life (GameOfLife): Instancia del juego.
        cell_size (int, optional): Tamaño de cada celda en píxeles. Aleatorio si None.
        alive_color (tuple, optional): Color de las celdas vivas (RGB). Aleatorio si None.
        dead_color (tuple, optional): Color de las celdas muertas (RGB). Aleatorio si None.
        grid_color (tuple): Color de las líneas de la cuadrícula (RGB).
        fps (int, optional): Cuadros por segundo para la animación. Aleatorio si None.
        show_stats (bool): Si True, muestra estadísticas minimalistas.
    """
    pygame.init()

    # Asignar valores por defecto o aleatorios
    cell_size = cell_size or random.choice(range(5, 21))
    alive_color, dead_color = alive_color or generate_color_pair()
    fps = fps or random.choice(range(5, 31))

    # Dimensiones de la ventana
    width, height = game_of_life.cols * cell_size, game_of_life.rows * cell_size
    screen = pygame.display.set_mode((width, height))
    pygame.display.set_caption("Game of Life")

    # Fuente para estadísticas (si están habilitadas)
    font = pygame.font.SysFont("consolas", 20)

    # Reloj para controlar el framerate
    clock = pygame.time.Clock()

    running = True
    paused = False
    generation = 0
    previous_boards = []
    loop_counter = 0
    max_loops = 60  # Límite de iteraciones en caso de estancamiento

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:  # Pausar/Reanudar
                    paused = not paused
                elif event.key == pygame.K_r:  # Reiniciar
                    game_of_life.board = np.random.randint(2, size=(game_of_life.rows, game_of_life.cols))
                    generation = 0
                    previous_boards = []
                    loop_counter = 0

        if not paused:
            # Avanzar un paso en la simulación
            game_of_life.step()
            generation += 1

            # Verificar ciclos o estancamiento
            board_tuple = tuple(map(tuple, game_of_life.board))
            if len(previous_boards) > 0 and board_tuple == previous_boards[-1]:
                # Si el tablero actual es igual al anterior, incrementar contador
                loop_counter += 1
            elif len(previous_boards) > 1 and board_tuple == previous_boards[-2]:
                # Si el tablero alterna con el anterior al último
                loop_counter += 1
            else:
                # Reiniciar contador si no hay ciclo
                loop_counter = 0

            if loop_counter >= max_loops:
                alive_color, dead_color = invert_colors(alive_color, dead_color)
                show_stats = True  # Forzar mostrar estadísticas
                running = False  # Terminar ejecución después del countdown

            previous_boards.append(board_tuple)
            if len(previous_boards) > 10:  # Mantener un historial limitado
                previous_boards.pop(0)

        # Dibujar el estado actual del tablero
        screen.fill(grid_color)  # Fondo con color de cuadrícula
        for row in range(game_of_life.rows):
            for col in range(game_of_life.cols):
                color = alive_color if game_of_life.board[row, col] == 1 else dead_color
                pygame.draw.rect(
                    screen,
                    color,
                    pygame.Rect(
                        col * cell_size, row * cell_size, cell_size, cell_size
                    )
                )

        # Mostrar estadísticas si está habilitado
        if show_stats:
            alive_cells = np.sum(game_of_life.board)
            stats_text = f"Gen: {generation} | Alive: {alive_cells}"
            stats_surface = font.render(stats_text, True, (255, 255, 255))
            screen.blit(stats_surface, (10, 10))

        pygame.display.flip()
        clock.tick(fps)

    pygame.quit()


pygame 2.6.1 (SDL 2.30.10, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


In [4]:
import os
import sys
import numpy as np
import game_of_life
import visualization
import random

# # Asegurar que /src esté en el sys.path para los imports
# src_path = os.path.abspath(os.path.join(os.path.dirname(__file__), "src"))
# if src_path not in sys.path:
#     sys.path.append(src_path)

def generate_random_board(rows, cols, seed=None):
    """
    Genera un tablero aleatorio de dimensiones especificadas.

    Args:
        rows (int): Número de filas del tablero.
        cols (int): Número de columnas del tablero.
        seed (int, optional): Semilla para reproducibilidad.

    Returns:
        numpy.ndarray: Tablero inicial aleatorio.
    """
    np.random.seed(seed)
    print(f"Generando un tablero aleatorio de {rows}x{cols} con semilla {seed}.")
    return np.random.randint(2, size=(rows, cols))

def generate_preset_board(preset=None, rows=5, cols=5):
    """
    Genera tableros predefinidos basados en patrones clásicos o selecciona uno aleatoriamente.

    Args:
        preset (str, optional): Nombre del patrón ("block", "blinker", "glider", "random"), o None para seleccionar "random".
        rows (int, optional): Número de filas para el tablero aleatorio.
        cols (int, optional): Número de columnas para el tablero aleatorio.

    Returns:
        numpy.ndarray: Tablero inicial con el patrón seleccionado.
    """
    patterns = {
        "block": np.array([
            [1, 1],
            [1, 1]
        ]),
        "blinker": np.array([
            [0, 1, 0],
            [0, 1, 0],
            [0, 1, 0]
        ]),
        "glider": np.array([
            [0, 1, 0],
            [0, 0, 1],
            [1, 1, 1]
        ]),
        "random": lambda: generate_random_board(rows, cols)  # Generar tablero aleatorio con dimensiones especificadas
    }

    if preset is None:
        preset = "random"
        print(f"Preset no especificado. Seleccionando: {preset}.")

    print(f"Seleccionando un patrón predefinido: {preset}.")

    if preset not in patterns:
        raise ValueError(f"Preset '{preset}' no está definido.")

    if preset == "random":
        return patterns[preset]()  # Genera dinámicamente un tablero aleatorio con las dimensiones dadas

    # Ajustar el patrón predefinido al tamaño especificado (centrado en el tablero)
    pattern = patterns[preset]
    board = np.zeros((rows, cols), dtype=int)
    start_row = (rows - pattern.shape[0]) // 2
    start_col = (cols - pattern.shape[1]) // 2
    board[start_row:start_row + pattern.shape[0], start_col:start_col + pattern.shape[1]] = pattern
    return board

def run_simulation(initial_state, steps=10):
    """
    Ejecuta una simulación del Game of Life.

    Args:
        initial_state (numpy.ndarray): Tablero inicial.
        steps (int): Número de pasos a ejecutar.

    Returns:
        GameOfLife: Objeto del juego tras la simulación.
    """
    print(f"Ejecutando simulación con {steps} pasos.")
    game = GameOfLife(initial_state=initial_state, steps=steps)
    game.run()
    return game

def test_simulation(rows=None, cols=None, steps=None, preset=None, seed=None, visualize=False):
    """
    Ejecuta una simulación unitaria con parámetros ajustables o aleatorizados y reporta resultados.

    Args:
        rows (int, optional): Número de filas del tablero.
        cols (int, optional): Número de columnas del tablero.
        steps (int, optional): Número de pasos a ejecutar.
        preset (str, optional): Patrón predefinido ("block", "blinker", "glider", "random") o None para seleccionar "random".
        seed (int, optional): Semilla para reproducibilidad.
        visualize (bool, optional): Si True, genera una visualización.

    Returns:
        dict: Resumen de la simulación.
    """
    # Aleatorizar parámetros si no se proporcionan
    rows = rows or random.choice(range(10, 101))
    cols = cols or random.choice(range(10, 101))
    steps = steps or random.choice(range(1, 501))

    if preset is None:
        preset = "random"
        print(f"Preset no especificado. Seleccionando: {preset}")
        initial_state = generate_preset_board(preset, rows, cols)
    elif preset == "random":
        seed = seed or random.randint(0, 1000000)  # Generar semilla aleatoria si no se proporciona
        initial_state = generate_random_board(rows, cols, seed)
    else:
        initial_state = generate_preset_board(preset, rows, cols)

    game = run_simulation(initial_state, steps)

    if visualize:
        print("Iniciando visualización...")
        visualize_game(game_of_life=game)

    summary = {
        "Dimensions": f"{rows} x {cols}",
        "Steps Executed": f"{game.execution.step_count}/{steps}",
        "Execution Time": f"{game.execution.execution_time:.2f} seconds",
        "Max Alive Cells": game.execution.max_alive_cells,
        "Min Alive Cells": game.execution.min_alive_cells,
        "Loop Detected": "Yes" if game.execution.loop_detected else "No",
        "Loop Length": game.execution.loop_length if game.execution.loop_detected else "N/A",
        "Seed": seed,
    }

    return summary

# # Ejemplo de ejecución directa
# if __name__ == "__main__":
#     report = test_simulation(visualize=True)
#     print("\n--- Resumen de la Simulación ---")
#     for key, value in report.items():
#         print(f"{key}: {value}")


In [5]:
help(game_of_life)

Help on module game_of_life:

NAME
    game_of_life

CLASSES
    builtins.object
        Execution
        GameOfLife

    class Execution(builtins.object)
     |  Execution(dimensions, steps, initial_state, seed=None)
     |
     |  Methods defined here:
     |
     |  __init__(self, dimensions, steps, initial_state, seed=None)
     |      Class to store and manage metadata and statistics for a Game of Life execution.
     |
     |      Args:
     |          dimensions (tuple): Dimensions of the board (rows, columns).
     |          steps (int): Maximum number of steps to execute.
     |          initial_state (numpy.ndarray): Initial state of the board.
     |          seed (int, optional): Seed used for reproducibility.
     |
     |  finalize(self, step_count, execution_time, loop_detected=False, loop_length=0)
     |      Finalizes the execution by recording final statistics.
     |
     |      Args:
     |          step_count (int): Total number of steps executed.
     |        

In [6]:
help(visualization)

Help on module visualization:

NAME
    visualization

FUNCTIONS
    generate_color_pair()
        Genera un par de colores predefinidos de una biblioteca estilística.

        Returns:
            tuple: (alive_color, dead_color) en formato RGB.

    invert_colors(alive_color, dead_color)
        Invierte los colores vivos y muertos.

        Args:
            alive_color (tuple): Color actual de las celdas vivas.
            dead_color (tuple): Color actual de las celdas muertas.

        Returns:
            tuple: Nuevos colores invertidos (alive_color, dead_color).

    visualize_game(game_of_life, cell_size=None, alive_color=None, dead_color=None, grid_color=(50, 50, 50), fps=None, show_stats=False)
        Visualiza la simulación del Game of Life usando pygame, con detección de bucles largos.

        Args:
            game_of_life (GameOfLife): Instancia del juego.
            cell_size (int, optional): Tamaño de cada celda en píxeles. Aleatorio si None.
            alive_color

In [7]:
help(test_simulation)

Help on function test_simulation in module __main__:

test_simulation(rows=None, cols=None, steps=None, preset=None, seed=None, visualize=False)
    Ejecuta una simulación unitaria con parámetros ajustables o aleatorizados y reporta resultados.

    Args:
        rows (int, optional): Número de filas del tablero.
        cols (int, optional): Número de columnas del tablero.
        steps (int, optional): Número de pasos a ejecutar.
        preset (str, optional): Patrón predefinido ("block", "blinker", "glider", "random") o None para seleccionar "random".
        seed (int, optional): Semilla para reproducibilidad.
        visualize (bool, optional): Si True, genera una visualización.

    Returns:
        dict: Resumen de la simulación.



In [11]:
from game_of_life import GameOfLife

test_simulation(visualize=True)

Preset no especificado. Seleccionando: random
Seleccionando un patrón predefinido: random.
Generando un tablero aleatorio de 74x52 con semilla None.
Ejecutando simulación con 171 pasos.
Iniciando visualización...


{'Dimensions': '74 x 52',
 'Steps Executed': '171/171',
 'Execution Time': '0.22 seconds',
 'Max Alive Cells': 1043,
 'Min Alive Cells': 100,
 'Loop Detected': 'No',
 'Loop Length': 'N/A',
 'Seed': None}

# new game engine con viz

In [12]:
import numpy as np
from datetime import datetime
import platform
import cpuinfo
from visualization import visualize_game

class Execution:
    def __init__(self, dimensions, steps, initial_state, seed=None):
        """
        Class to store and manage metadata and statistics for a Game of Life execution.

        Args:
            dimensions (tuple): Dimensions of the board (rows, columns).
            steps (int): Maximum number of steps to execute.
            initial_state (numpy.ndarray): Initial state of the board.
            seed (int, optional): Seed used for reproducibility.
        """
        self.dimensions = dimensions
        self.steps = steps
        self.initial_state = initial_state
        self.seed = seed if seed is not None else np.random.randint(0, 1000000)
        self.timestamp = datetime.now()  # Timestamp for the start of the execution
        self.step_count = 0
        self.alive_cells_stats = []  # Percentage of alive cells per step
        self.max_alive_cells = 0
        self.min_alive_cells = np.prod(dimensions)
        self.execution_time = 0
        self.processor_info = cpuinfo.get_cpu_info()
        self.loop_detected = False
        self.loop_length = 0
        self.operations_count = 0

    def update_stats(self, board):
        """
        Updates the statistics of the execution for each step.

        Args:
            board (numpy.ndarray): Current board state after a step.
        """
        alive_cells = np.sum(board)
        total_cells = self.dimensions[0] * self.dimensions[1]
        alive_percentage = alive_cells / total_cells * 100
        self.alive_cells_stats.append(alive_percentage)
        self.max_alive_cells = max(self.max_alive_cells, alive_cells)
        self.min_alive_cells = min(self.min_alive_cells, alive_cells)
        self.operations_count += 1

    def finalize(self, step_count, execution_time, loop_detected=False, loop_length=0):
        """
        Finalizes the execution by recording final statistics.

        Args:
            step_count (int): Total number of steps executed.
            execution_time (float): Total execution time in seconds.
            loop_detected (bool): Whether a loop was detected.
            loop_length (int): Length of the detected loop.
        """
        self.step_count = step_count
        self.execution_time = execution_time
        self.loop_detected = loop_detected
        self.loop_length = loop_length

    def get_loop_info(self):
        """
        Returns loop detection status and its length.

        Returns:
            tuple: (loop_detected, loop_length)
        """
        return self.loop_detected, self.loop_length

    def summary(self):
        """
        Generates a user-friendly summary of the execution statistics.

        Returns:
            str: A formatted string summarizing the execution.
        """
        return f"""
        Game of Life Execution Summary
        --------------------------------
        Dimensions: {self.dimensions[0]} x {self.dimensions[1]}
        Steps Executed: {self.step_count}/{self.steps}
        Execution Time: {self.execution_time:.2f} seconds
        Max Alive Cells: {self.max_alive_cells}
        Min Alive Cells: {self.min_alive_cells}
        Operations Count: {self.operations_count}
        Loop Detected: {'Yes' if self.loop_detected else 'No'}
        Loop Length: {self.loop_length if self.loop_detected else 'N/A'}
        Timestamp: {self.timestamp.strftime('%Y-%m-%d %H:%M:%S')}
        Seed: {self.seed if self.seed is not None else 'Randomly Generated'}
        Processor: {self.processor_info.get('brand_raw', 'Unknown Processor')}
        System: {platform.system()} {platform.architecture()[0]}
        """

    def to_dict(self):
        """
        Converts the metadata into a dictionary for export.

        Returns:
            dict: A dictionary containing metadata and statistics.
        """
        return {
            "dimensions": self.dimensions,
            "steps": self.steps,
            "step_count": self.step_count,
            "execution_time": self.execution_time,
            "max_alive_cells": self.max_alive_cells,
            "min_alive_cells": self.min_alive_cells,
            "alive_cells_stats": self.alive_cells_stats,
            "seed": self.seed,
            "timestamp": self.timestamp.isoformat(),
            "processor": self.processor_info.get('brand_raw', 'Unknown Processor'),
            "architecture": platform.architecture()[0],
            "system": platform.system(),
            "processor_name": platform.processor(),
            "loop_detected": self.loop_detected,
            "loop_length": self.loop_length,
            "operations_count": self.operations_count
        }

class GameOfLife:
    def __init__(self, dimensions=(10, 10), steps=0, initial_state=None, seed=None):
        """
        Class implementing Conway's Game of Life simulation.

        Args:
            dimensions (tuple): Dimensions of the board (rows, columns).
            steps (int): Number of steps to run the simulation.
            initial_state (numpy.ndarray, optional): Custom initial board state.
            seed (int, optional): Seed for reproducibility.
        """
        self.rows, self.cols = dimensions
        self.steps = steps
        np.random.seed(seed)
        self.seed = seed
        self.board = (
            initial_state
            if initial_state is not None
            else np.random.randint(2, size=(self.rows, self.cols))
        )
        self.execution = Execution(
            dimensions=dimensions, steps=steps, initial_state=self.board.copy(), seed=seed
        )

    def count_neighbors(self, board):
        """
        Counts the number of alive neighbors for each cell.

        Args:
            board (numpy.ndarray): Current state of the board.

        Returns:
            numpy.ndarray: Array with neighbor counts for each cell.
        """
        neighbors = (
            np.roll(np.roll(board, 1, axis=0), 1, axis=1) +
            np.roll(np.roll(board, 1, axis=0), -1, axis=1) +
            np.roll(np.roll(board, -1, axis=0), 1, axis=1) +
            np.roll(np.roll(board, -1, axis=0), -1, axis=1) +
            np.roll(board, 1, axis=0) +
            np.roll(board, -1, axis=0) +
            np.roll(board, 1, axis=1) +
            np.roll(board, -1, axis=1)
        )
        return neighbors

    def step(self):
        """
        Executes a single step of the simulation, updating the board state.
        """
        neighbors = self.count_neighbors(self.board)
        self.board = (
            (self.board & (neighbors == 2)) | (neighbors == 3)
        ).astype(int)
        self.execution.update_stats(self.board)

    def run(self):
        """
        Runs the simulation for the specified number of steps.
        """
        import time
        start_time = time.time()
        previous_boards = []
        loop_detected = False
        loop_length = 0

        for step in range(self.steps):
            self.step()

            # Check for loops
            board_tuple = tuple(map(tuple, self.board))
            if board_tuple in previous_boards:
                loop_detected = True
                loop_length = len(previous_boards) - previous_boards.index(board_tuple)
                break
            previous_boards.append(board_tuple)
            if len(previous_boards) > 100:  # Limit history size
                previous_boards.pop(0)

        end_time = time.time()
        self.execution.finalize(step + 1, end_time - start_time, loop_detected, loop_length)

    def get_execution_stats(self):
        """
        Retrieves the execution statistics.

        Returns:
            str: Formatted summary of the execution statistics.
        """
        return self.execution.summary()

# Centralized interface for running and visualizing the game
def run_and_visualize(dimensions, steps, seed=None):
    """
    Centralized method to run and visualize a Game of Life simulation.

    Args:
        dimensions (tuple): Dimensions of the board (rows, columns).
        steps (int): Number of steps to run the simulation.
        seed (int, optional): Seed for reproducibility.
    """
    game = GameOfLife(dimensions=dimensions, steps=steps, seed=seed)
    game.run()
    visualize_game(game)


In [19]:
import game_of_life 
help(game_of_life)


Help on module game_of_life:

NAME
    game_of_life

CLASSES
    builtins.object
        Execution
        GameOfLife

    class Execution(builtins.object)
     |  Execution(dimensions, steps, initial_state, seed=None)
     |
     |  Methods defined here:
     |
     |  __init__(self, dimensions, steps, initial_state, seed=None)
     |      Class to store and manage metadata and statistics for a Game of Life execution.
     |
     |      Args:
     |          dimensions (tuple): Dimensions of the board (rows, columns).
     |          steps (int): Maximum number of steps to execute.
     |          initial_state (numpy.ndarray): Initial state of the board.
     |          seed (int, optional): Seed used for reproducibility.
     |
     |  finalize(self, step_count, execution_time, loop_detected=False, loop_length=0)
     |      Finalizes the execution by recording final statistics.
     |
     |      Args:
     |          step_count (int): Total number of steps executed.
     |        

In [21]:

# Ejecutar y visualizar una simulación
run_and_visualize(dimensions=(50, 50), steps=200, seed=42)


In [22]:
import pygame
import numpy as np
import random
from PIL import Image

def generate_color_pair():
    """
    Genera un par de colores predefinidos de una biblioteca estilística.

    Returns:
        tuple: (alive_color, dead_color) en formato RGB.
    """
    color_palettes = [
        ((0, 255, 0), (0, 0, 0)),  # Verde vivo sobre negro
        ((0, 0, 255), (10, 10, 30)),  # Azul vivo sobre azul oscuro
        ((255, 165, 0), (30, 15, 5)),  # Naranja sobre marrón oscuro
        ((255, 0, 0), (50, 0, 0)),  # Rojo sobre rojo oscuro
        ((255, 255, 255), (0, 0, 0)),  # Blanco sobre negro
        ((255, 223, 186), (34, 34, 59))  # Tono pastel sobre azul profundo
    ]
    return random.choice(color_palettes)

def invert_colors(alive_color, dead_color):
    """
    Invierte los colores vivos y muertos.

    Args:
        alive_color (tuple): Color actual de las celdas vivas.
        dead_color (tuple): Color actual de las celdas muertas.

    Returns:
        tuple: Nuevos colores invertidos (alive_color, dead_color).
    """
    return dead_color, alive_color

def capture_frame(surface):
    """
    Captura el contenido de la ventana de pygame como una imagen de PIL.

    Args:
        surface (pygame.Surface): Superficie de la ventana de pygame.

    Returns:
        PIL.Image: Imagen capturada de la ventana.
    """
    data = pygame.image.tostring(surface, "RGB")
    width, height = surface.get_size()
    return Image.frombytes("RGB", (width, height), data)

def visualize_game(
    game_of_life,
    cell_size=None,
    alive_color=None,
    dead_color=None,
    grid_color=(50, 50, 50),
    fps=None,
    show_stats=False,
    save_as_gif=False,
    gif_path="game_of_life.gif"
):
    """
    Visualiza la simulación del Game of Life usando pygame, con captura de cuadros y exportación a GIF.

    Args:
        game_of_life (GameOfLife): Instancia del juego.
        cell_size (int, optional): Tamaño de cada celda en píxeles. Aleatorio si None.
        alive_color (tuple, optional): Color de las celdas vivas (RGB). Aleatorio si None.
        dead_color (tuple, optional): Color de las celdas muertas (RGB). Aleatorio si None.
        grid_color (tuple): Color de las líneas de la cuadrícula (RGB).
        fps (int, optional): Cuadros por segundo para la animación. Aleatorio si None.
        show_stats (bool): Si True, muestra estadísticas minimalistas.
        save_as_gif (bool): Si True, guarda la simulación como un archivo GIF.
        gif_path (str): Ruta para guardar el GIF.
    """
    pygame.init()

    # Asignar valores por defecto o aleatorios
    cell_size = cell_size or random.choice(range(5, 21))
    alive_color, dead_color = alive_color or generate_color_pair()
    fps = fps or random.choice(range(5, 31))

    # Dimensiones de la ventana
    width, height = game_of_life.cols * cell_size, game_of_life.rows * cell_size
    screen = pygame.display.set_mode((width, height))
    pygame.display.set_caption("Game of Life")

    # Fuente para estadísticas (si están habilitadas)
    font = pygame.font.SysFont("consolas", 20)

    # Reloj para controlar el framerate
    clock = pygame.time.Clock()

    running = True
    paused = False
    generation = 0
    previous_boards = []
    loop_counter = 0
    max_loops = 60  # Límite de iteraciones en caso de estancamiento

    frames = []  # Almacenar cuadros para crear el GIF

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:  # Pausar/Reanudar
                    paused = not paused
                elif event.key == pygame.K_r:  # Reiniciar
                    game_of_life.board = np.random.randint(2, size=(game_of_life.rows, game_of_life.cols))
                    generation = 0
                    previous_boards = []
                    loop_counter = 0

        if not paused:
            # Avanzar un paso en la simulación
            game_of_life.step()
            generation += 1

            # Verificar ciclos o estancamiento
            board_tuple = tuple(map(tuple, game_of_life.board))
            if len(previous_boards) > 0 and board_tuple == previous_boards[-1]:
                # Si el tablero actual es igual al anterior, incrementar contador
                loop_counter += 1
            elif len(previous_boards) > 1 and board_tuple == previous_boards[-2]:
                # Si el tablero alterna con el anterior al último
                loop_counter += 1
            else:
                # Reiniciar contador si no hay ciclo
                loop_counter = 0

            if loop_counter >= max_loops:
                alive_color, dead_color = invert_colors(alive_color, dead_color)
                show_stats = True  # Forzar mostrar estadísticas
                running = False  # Terminar ejecución después del countdown

            previous_boards.append(board_tuple)
            if len(previous_boards) > 10:  # Mantener un historial limitado
                previous_boards.pop(0)

        # Dibujar el estado actual del tablero
        screen.fill(grid_color)  # Fondo con color de cuadrícula
        for row in range(game_of_life.rows):
            for col in range(game_of_life.cols):
                color = alive_color if game_of_life.board[row, col] == 1 else dead_color
                pygame.draw.rect(
                    screen,
                    color,
                    pygame.Rect(
                        col * cell_size, row * cell_size, cell_size, cell_size
                    )
                )

        # Capturar cuadro si se va a guardar como GIF
        if save_as_gif:
            frames.append(capture_frame(screen))

        # Mostrar estadísticas si está habilitado
        if show_stats:
            alive_cells = np.sum(game_of_life.board)
            stats_text = f"Gen: {generation} | Alive: {alive_cells}"
            stats_surface = font.render(stats_text, True, (255, 255, 255))
            screen.blit(stats_surface, (10, 10))

        pygame.display.flip()
        clock.tick(fps)

    pygame.quit()

    # Guardar el GIF si es necesario
    if save_as_gif and frames:
        frames[0].save(
            gif_path,
            save_all=True,
            append_images=frames[1:],
            duration=1000 // fps,
            loop=0
        )
        print(f"GIF guardado en {gif_path}")


In [1]:
from game_of_life import GameOfLife
from visualization import visualize_game

game = GameOfLife(dimensions=(20, 20), steps=100, seed=42)
visualize_game(game, save_as_gif=True, gif_path="simulation.gif")


<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


pygame 2.6.1 (SDL 2.30.10, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
from game_of_life import GameOfLife
from visualization import visualize_game

game = GameOfLife(dimensions=(20, 20), steps=100, seed=42)
visualize_game(game, save_as_gif=True, gif_path="simulation.gif", verbose=True)



--- Simulation Parameters ---
Cell Size: 13
Alive Color: (0, 0, 255)
Dead Color: (10, 10, 30)
Grid Color: (50, 50, 50)
FPS: 9
Save as GIF: True
GIF Path: simulation.gif
GIF guardado en simulation.gif
